# Science Case #1a: Black Holes and CO gas

Expanded example from the report, also shows how to add a new column with the linear scale in parsecs, and compare the VLSR and our rdz lookup table.


In [1]:
from astroquery.admit import ADMIT
import pandas as pd

In [2]:
a = ADMIT()
a.check()

Found  /home/teuben/ALMA/study7/query/admit.db
Checking db.... 0
71 71 71
Database version: 27-feb-2022. core.py version: 26-feb-2022
header    : 1 entries
alma      : 131 entries
win       : 130 entries
lines     : 33 entries
sources   : 801 entries


### Find projects about black holes where CO was observed.

Note: we avoid B and H for because the underlying SQL search is case sensitive

In [3]:
p = a.query(formula="CO",project_abstract="*lack*ole*")

select * from alma inner join win on (win.a_id = alma.id)  inner join lines on (lines.w_id = win.id )  WHERE lines.formula='CO' AND alma.project_abstract LIKE '%lack%ole%' 



In [4]:
p

,id,obs_id,target_name,s_ra,s_dec,frequency,s_resolution,t_min,cont_sensitivity_bandwidth,sensitivity_10kms,...,id,w_id,formula,transition,restfreq,vmin,vmax,mom0flux,mom1peak,mom2peak
0,6,uid://A001/X1288/Xba2,NGC3049,148.706950,9.271113,229.424396,0.332481,58379.538206,0.015700,0.472535,...,5,6,CO,2-1,230.538,-51.54370,105.41400,2698.220,1475.480,20.86430
1,10,uid://A001/X1288/Xba6,NGC3504,165.796595,27.972404,229.371212,0.034121,58050.543677,0.013024,0.410236,...,7,10,CO,2-1,230.538,-100.23700,88.66320,3388.780,1516.190,34.98030
2,14,uid://A001/X1288/Xba8,NGC3504,165.796595,27.972404,229.371216,0.200656,58119.297186,0.021794,0.686479,...,9,14,CO,2-1,230.538,-148.71500,167.81400,18313.700,1547.690,58.09510
3,15,uid://A001/X1288/Xbae,NGC3593,168.654583,12.817675,230.055098,0.289034,58384.714766,0.038697,0.604888,...,10,15,CO,2-1,230.538,-136.84900,138.02900,11534.200,639.522,70.28280
4,26,uid://A001/X1288/Xbc0,NGC4414,186.612807,31.223649,229.988814,0.223839,58123.420166,0.015109,0.455252,...,17,26,CO,2-1,230.538,-168.82400,178.66800,9306.800,710.181,62.40980
5,30,uid://A001/X1288/Xbc4,NGC5253,204.983216,-31.640107,230.225275,0.074072,58076.500165,0.010960,0.344422,...,20,30,CO,2-1,230.538,-40.36360,6.68647,436.564,387.738,10.90870
6,34,uid://A001/X1288/Xbc6,NGC5253,204.983216,-31.640107,230.225281,0.258194,58382.847846,0.015421,0.466701,...,21,34,CO,2-1,230.538,-46.70160,46.13340,894.771,397.203,15.01770
7,42,uid://A001/X1296/X6f3,5-NGC3773,174.553958,12.111899,229.781272,0.299770,58384.680357,0.018072,0.568947,...,27,42,CO,2-1,230.538,-9.50778,19.79620,323.716,980.293,6.99879


In [5]:
ci=['obs_id','target_name','s_resolution', 'vlsr']
print(p[ci])

                  obs_id target_name  s_resolution         vlsr
0  uid://A001/X1288/Xba2     NGC3049      0.332481  1447.989431
1  uid://A001/X1288/Xba6     NGC3504      0.034121  1521.106704
2  uid://A001/X1288/Xba8     NGC3504      0.200656  1521.106704
3  uid://A001/X1288/Xbae     NGC3593      0.289034   628.000000
4  uid://A001/X1288/Xbc0     NGC4414      0.223839   715.580697
5  uid://A001/X1288/Xbc4     NGC5253      0.074072   401.856222
6  uid://A001/X1288/Xbc6     NGC5253      0.258194   401.856222
7  uid://A001/X1296/X6f3   5-NGC3773      0.299770   978.913173


This looks interesting. But we like to know the resolution in physical pc, not in arcsec. 

Note: the current **s_resolution** is in arcsec, which is in error, it should have been degrees. This is a fix the archive group will make sometime in 2022


In [6]:
H0 = 75
ascale = 1.0      # this will be 3600 once s_resolution is in degrees again

p['size_pc'] = p['s_resolution']/ascale * p['vlsr']/H0 * 1e6/206265

In [7]:
ci=['obs_id','target_name','s_resolution', 'vlsr','size_pc']
print(p[ci])

                  obs_id target_name  s_resolution         vlsr    size_pc
0  uid://A001/X1288/Xba2     NGC3049      0.332481  1447.989431  31.120376
1  uid://A001/X1288/Xba6     NGC3504      0.034121  1521.106704   3.355025
2  uid://A001/X1288/Xba8     NGC3504      0.200656  1521.106704  19.729870
3  uid://A001/X1288/Xbae     NGC3593      0.289034   628.000000  11.733352
4  uid://A001/X1288/Xbc0     NGC4414      0.223839   715.580697  10.353969
5  uid://A001/X1288/Xbc4     NGC5253      0.074072   401.856222   1.924158
6  uid://A001/X1288/Xbc6     NGC5253      0.258194   401.856222   6.707033
7  uid://A001/X1296/X6f3   5-NGC3773      0.299770   978.913173  18.969017


We don't have information here on the expected black hole mass, which should be ~0.01 * bulge_mass 

In [8]:
p[ci].describe()

,s_resolution,vlsr,size_pc
count,8.000000,8.000000,8.000000
mean,0.214021,952.051144,12.986600
std,0.107696,487.073056,9.806514
min,0.034121,401.856222,1.924158
25%,0.169010,571.464055,5.869031
50%,0.241016,847.246935,11.043660
75%,0.291718,1466.268749,19.159231
max,0.332481,1521.106704,31.120376


## Accuracy of VLSR?  

The VLSR that ADMIT uses is not reliable. Since the PI information of VLSR is optional, and not made available in the fits headers, ADMIT uses the RESTFREQ in the fits header, and compares that to the frequency in the middle of the spw. The PI can easily use another algorithm, though often this method works.

We are using a table lookup provided by Felix Stoehr (ESO) that we added as an option to a special query in out admit astroquery. This query_rdz() returns the 'z' of the object, e.g.

In [9]:
a.query_rdz('uid://A001/X1288/Xba6','NGC3504')

Found 49765 entries in rdz
Searching for  uid://A001/X1288/Xba6 NGC3504
uid://A001/X1288/Xba6.source.NGC3504|165.796629167|27.9724361111|0.005104|NGC  3504|



0.005104

an alternative method would be a lookup via NED , at least for galaxies. Here's the example for NGC3504

In [10]:
from astroquery.ipac.ned import Ned
Ned.query_object("NGC3504")

No.,Object Name,RA,DEC,Type,Velocity,Redshift,Redshift Flag,Magnitude and Filter,Separation,References,Notes,Photometry Points,Positions,Redshift Points,Diameter Points,Associations
,,degrees,degrees,,km / s,,,,arcmin,,,,,,,
int32,bytes30,float64,float64,object,float64,float64,object,object,float64,int32,int32,int32,int32,int32,int32,int32
1,NGC 3504,165.79671,27.9725,G,1525.0,0.005087,,11.4B,--,529,23,133,49,27,8,0


Looping over all galaxies in our query, we grab the redshift in a new array, and add it to the dataframe.

In [11]:
vz = np.zeros(len(p)) 
for i in range(len(p)):
    vz[i] =  a.query_rdz(p['obs_id'][i],p['target_name'][i])

Searching for  uid://A001/X1288/Xba2 NGC3049
uid://A001/X1288/Xba2.source.NGC3049|148.706517455|9.2710935803|0.00485400017|NGC 3049|

Searching for  uid://A001/X1288/Xba6 NGC3504
uid://A001/X1288/Xba6.source.NGC3504|165.796629167|27.9724361111|0.005104|NGC  3504|

Searching for  uid://A001/X1288/Xba8 NGC3504
uid://A001/X1288/Xba8.source.NGC3504|165.796629167|27.9724361111|0.005104|NGC  3504|

Searching for  uid://A001/X1288/Xbae NGC3593
uid://A001/X1288/Xbae.source.NGC3593|168.654166667|12.8176666667|0.00209499989|NGC 3593|

Searching for  uid://A001/X1288/Xbc0 NGC4414
uid://A001/X1288/Xbc0.source.NGC4414|186.612870833|31.2235444444|0.002395|NGC  4414|

Searching for  uid://A001/X1288/Xbc4 NGC5253
uid://A001/X1288/Xbc4.source.NGC5253|204.983179583|-31.6401077778|0.00135799998|NGC 5253|

Searching for  uid://A001/X1288/Xbc6 NGC5253
Searching for  uid://A001/X1296/X6f3 5-NGC3773
uid://A001/X1296/X6f3.source.5-NGC3773|174.553647831|12.1120476073|0.00327600003|NGC 3773|



In [12]:
p['rdz'] = vz*300000

In [13]:
ci=['obs_id','target_name', 'vlsr','rdz','size_pc']
print(p[ci])

                  obs_id target_name         vlsr          rdz    size_pc
0  uid://A001/X1288/Xba2     NGC3049  1447.989431  1456.200051  31.120376
1  uid://A001/X1288/Xba6     NGC3504  1521.106704  1531.200000   3.355025
2  uid://A001/X1288/Xba8     NGC3504  1521.106704  1531.200000  19.729870
3  uid://A001/X1288/Xbae     NGC3593   628.000000   628.499967  11.733352
4  uid://A001/X1288/Xbc0     NGC4414   715.580697   718.500000  10.353969
5  uid://A001/X1288/Xbc4     NGC5253   401.856222   407.399994   1.924158
6  uid://A001/X1288/Xbc6     NGC5253   401.856222          NaN   6.707033
7  uid://A001/X1296/X6f3   5-NGC3773   978.913173   982.800009  18.969017


One of the obsid's for NGC5253 was apparently not in the "rdz" database, luckily another one did.